First lets find the bryk

In [83]:
import nest_asyncio
import asyncio
import struct
from bleak import discover, BleakClient # BleakScanner
nest_asyncio.apply()

class PhysBryk(object):
    
    def __init__(self):
        self._device = None
    
    def setDevice(self, ble_device):
        self._device = ble_device
        
    def getAddress(self):
        return self._device.address
        
bryk = PhysBryk()
        
async def run():
    print('Searching...')
    devices = await discover()
    for d in devices:
        print(d.name, d.address, d.rssi, d.metadata)
        if d.name == 'PhysBryk':
            bryk.setDevice(d)
    print('Search completed')
    
    if bryk:
        print(f'PhysBryk found @ {bryk.getAddress()}')
    else:
        print('No PhysBryk found')
    
loop = asyncio.get_event_loop()
loop.run_until_complete(run())

print(bryk)

Searching...
PhysBryk D8:95:3B:9E:D2:D6 -52 {'uuids': [], 'manufacturer_data': {}}
Unknown C8:69:CD:3B:5E:DC -78 {'uuids': [], 'manufacturer_data': {76: b'\t\x06\x03\x02\n\x86t\xc2'}}
Search completed
PhysBryk found @ D8:95:3B:9E:D2:D6


Now lets connect

In [89]:
bryk_addr = bryk.getAddress()

INTENSITY_UUID = 'a0d11001-0eaa-5b52-bc84-818888dc7dc5'

async def print_services(mac_addr: str):
#     device = await BleakScanner.find_device_by_address(mac_addr)
    async with BleakClient(mac_addr) as client:
        svcs = await client.get_services()
        print("Services:", svcs)
        for s in svcs:
            print(s)
            for c in s.characteristics: print(c)
        
        light_level = await client.read_gatt_char(INTENSITY_UUID)
        for reading in light_level: print(reading)
                

loop = asyncio.get_event_loop()
loop.run_until_complete(print_services(bryk_addr))

# a0d10301-0eaa-5b52-bc84-818888dc7dc5

Services: <bleak.backends.service.BleakGATTServiceCollection object at 0x000002BFFFC1C640>


AttributeError: 'list' object has no attribute 'items'